In [1]:
!pip install qiskit

In [ ]:
# unzipped already, no need to do again
!unzip pima-indians-diabetes-database.zip

pima-indians-diabetes-database.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  pima-indians-diabetes-database.zip
replace diabetes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# just in case someone wants to see the ugly comma seperated file
import csv
with open('/data/diabetes.csv', newline='') as File:  
    reader = csv.reader(File)
    for row in reader:
        print(row)

In [2]:
import numpy as np
data = np.genfromtxt('diabetes.csv', delimiter=',', names=True, case_sensitive=True)
data = np.array(data.tolist()) # was an array of arrays earlier
print(data.shape)
x = np.copy(data[:,0:8]) #features
y = np.copy(data[:,8]) #targets
features = 8 #or number of qubits which I kept equal to number of features in this case

(768, 9)


In [3]:
from sklearn.model_selection import train_test_split
import qiskit
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import one_against_rest, all_pairs
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
#from qiskit.aqua.input import ClassificationInput
#from qiskit.aqua import run_algorithm

In [4]:
#please and please again, use your own IBMQ token (can be generated easily), I have very few processes left for me.
key = '243ff4202efc1b45ad9282b9d6d524cdcccfc25561a86de37424b5ddd9aaa72969ea300500423019c48da10777e3241a5a45ff4a223439239cba4bed8d8709cc'

In [5]:
qiskit.IBMQ.save_account(key)
provider = qiskit.IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')

configrc.store_credentials:WARNING:2020-06-11 14:36:01,013: Credentials already present. Set overwrite=True to overwrite.


In [6]:
#using only first 100, if anyone is brave enough, train_test_split(x, y, test_size=0.2)
train_x, test_x, train_y, test_y = train_test_split(x[0:201], y[0:201], test_size=0.2)

training_data = {'A':train_x[train_y==0],'B':train_x[train_y==1]}
testing_data = {'A':test_x[test_y==0],'B':test_x[test_y==1]}

In [7]:
feature_map = SecondOrderExpansion(feature_dimension=features,depth=2,entanglement='full')
svm = QSVM(feature_map,training_data,testing_data)
shots = 1 
quantum_instance = QuantumInstance(backend,shots=shots,skip_qobj_validation=False)

/home/utkarsh/QC/qc/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The qiskit.aqua.components.feature_maps.SecondOrderExpansion object is deprecated as of 0.7.0 and will be removed no sooner than 3 months after the release. You should use qiskit.circuit.library.ZZFeatureMap instead.
  """Entry point for launching an IPython kernel.
/home/utkarsh/QC/qc/lib/python3.6/site-packages/qiskit/aqua/components/feature_maps/second_order_expansion.py:59: DeprecationWarning: The qiskit.aqua.components.feature_maps.PauliZExpansion class is deprecated as of 0.7.0 and will be removed no sooner than 3 months after the release. You should use qiskit.circuit.library.PauliFeatureMap instead.
  z_order=2, data_map_func=data_map_func)
/home/utkarsh/QC/qc/lib/python3.6/site-packages/qiskit/aqua/components/feature_maps/pauli_z_expansion.py:71: DeprecationWarning: The qiskit.aqua.components.feature_maps.PauliExpansion object is deprecated as of 0.7.0 and will be removed no sooner tha

In [8]:
%%time
result = svm.run(quantum_instance) 

CPU times: user 41min 37s, sys: 1min 3s, total: 42min 41s
Wall time: 1h 21min 56s


In [9]:
print(result)

{'kernel_matrix_training': array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]]), 'svm': {'alphas': array([ 42.41086737,  45.19225621,  39.38536136, 145.85999402,
        39.38536136,  39.38536136,  39.38536136, 256.98512649,
        39.38536136,  39.38536136,  39.38536136,   6.42690241,
       145.85999402,  39.38536136,  26.88946314,  39.38536136,
        39.38536136,  39.38536136,  39.38536136,  39.38536136,
        39.38536136,  39.38536136,  39.38536136,  28.17628991,
        39.38536136,  39.38536136,  39.38536136,  39.38536136,
        28.73247651, 161.00500605,  39.38536136,  28.73247651,
       145.85999402, 161.00500605,  27.06223944,  39.38536136,
        42.41086729,  39.38536136, 191.95026548,  26.88946314,
        39.38536136,  39.38536136,  42.41086729,  28.73247651,
        39.38536136,